<H1>Relationship between IL and price change in a stableswap pool</H1>


In [4]:
import sys
import os
import glob
import random
import copy
sys.path.append('../..')

from model import processing
from matplotlib import pyplot as plt

from model import run
from model import plot_utils as pu
from model.amm.omnipool_amm import OmnipoolState, cash_out_omnipool, value_assets, usd_price, lrna_price
from model.amm.stableswap_amm import StableSwapPoolState
from model.amm.agents import Agent
from model.amm.trade_strategies import omnipool_arbitrage, invest_all, price_sensitive_trading
from model.amm.global_state import GlobalState, fluctuate_prices, historical_prices

# same seed, same parameters = same simulation result
random.seed(42)
# price_list = processing.import_binance_prices(['BTC', 'ETH', 'DOT'], start_date='Jan 1 2023', days = 120)

assets = {
    'USDA': {'usd price': 1, 'weight': 0.5},
    'USDB': {'usd price': 1, 'weight': 0.5}
}

assert sum([t['weight'] for t in assets.values()]) == 1

initial_tvl = 1000000

initial_state = GlobalState(
    pools={
        'stableswap': StableSwapPoolState(
            tokens={
                tkn: initial_tvl * assets[tkn]['weight'] for tkn in assets
            },
            amplification=1000,
            trade_fee=0
        )
    },
    agents = {
        'LP': Agent(
            holdings={'USDA': 5100, 'USDB': 0},
            trade_strategy=invest_all('stableswap')
        ),
        'Trader': Agent(
            holdings={tkn: 10000000000000 for tkn in assets}
        )
    },
    external_market={tkn: assets[tkn]['usd price'] for tkn in assets},
    # archive_all=False
)


In [5]:
# steps = 201
# # price_min = 0.2
# # price_max = 5
# trade_size_min, trade_size_max = -1000000, 1000000
# step_factor = (trade_size_max / trade_size_min) ** (1 / (steps - 1))
# trade_sizes = [trade_size_min + i * (trade_size_max - trade_size_min) / steps for i in range(steps)]
# # [-trade_size_min * step_factor ** i for i in range(steps)][::-1] + [trade_size_min * step_factor ** i for i in range(steps)]

trade_size = 1000000

events = []

initial_holdings = copy.deepcopy(initial_state.agents['LP'].holdings)
initial_trader_holdings = copy.deepcopy(initial_state.agents['Trader'].holdings)
first_state = initial_state.copy()

events.append(
    run.run(initial_state, time_steps=1, silent=True)[-1].execute_swap(
        pool_id='stableswap',
        agent_id='LP',
        tkn_sell='USDA' if trade_size > 0 else 'USDB',
        tkn_buy='USDB' if trade_size > 0 else 'USDA',
        sell_quantity=max(abs(trade_size), 0),
    )
)

new_state = events[-1].copy()

# StableSwapPoolState.execute_remove_uniform(
#     state=event.pools['stableswap'],
#     agent=event.agents['LP'],
#     shares_removed=event.agents['LP'].holdings['stableswap']
# )

StableSwapPoolState.execute_remove_liquidity(
    state=new_state.pools['stableswap'],
    agent=new_state.agents['LP'],
    shares_removed=new_state.agents['LP'].holdings['stableswap'],
    tkn_remove='USDA'
)

final_state = new_state.copy()

trade_size2 = -trade_size

final_state.execute_swap(
        pool_id='stableswap',
        agent_id='LP',
        tkn_sell='USDA' if trade_size2 > 0 else 'USDB',
        tkn_buy='USDB' if trade_size2 > 0 else 'USDA',
        sell_quantity=abs(trade_size2),
    )



global state 
pools: 

    Stable Swap Pool: stableswap
    ********************************
    trade fee: 0
    shares: 1000000.0
    amplification constant: 1000
    tokens: (
    
        USDA
        quantity: 500000.0
        weight: 0.5
    
        USDB
        quantity: 500000.0
        weight: 0.5
    
    )
    error message:Agent has insufficient funds.

agents: 

    Agent: LP
    ********************************
    trade strategy: invest all (stableswap)
    holdings: (
    
        *USDA*: 5100.0
    
        *USDB*: 0
    
        *USD*: 0
    
        *stableswap*: 0.0
        price: 1.0000012679
    )
    
    Agent: Trader
    ********************************
    trade strategy: None
    holdings: (
    
        *USDA*: 10000000000000
    
        *USDB*: 10000000000000
    
        *USD*: 0
    )
    
market prices: 

        USDA: $1
        USDB: $1
        USD: $1

evolution function: None

In [6]:
final_prices = [1, final_state.pools['stableswap'].spot_price]
print(final_prices)

[1, 1.0]


In [7]:
print(initial_holdings)

{'USDA': 5100, 'USDB': 0}


In [8]:
print(final_state.agents['LP'].holdings)

{'USDA': 5100.0, 'USDB': 0, 'USD': 0, 'stableswap': 0.0}
